In [6]:
# TODO which one?
#git clone https://github.com/lucidrains/iTransformer.git
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from pathlib import Path

window_size = 96
pred_length = (96)

from utils import data_handling, training_functions, helpers
import config 

print("Import succesfull")

Import succesfull


# Benchmark all datasets used for transfer learning on iTransformer on its own.

After the sanity check, we will use 96h as the input and the prediction horizon. In the following we will use the RevIn normalization strategie to benchmark iTransformer on our transfer learning datasets to get an initial baseline. RevIn is used because if performed the best on our initial baseline. This baseline will be compared to an SARIMA implementation and an additional DL model.

Those results will also be used to evaluate the transfer-learning capability of iTransformer.

Because transfer-learning is a sensible solution for the cold-start problem, we also do benchmarks on iTransformer only trained on the first 10% of the train dataset. Beacuse all datasets are big enough for efficient training, using a small subset is a solutoin to get meaningfull insight in how much value can be created through transfer-learning. 

In [12]:
# use full train dataset for training or small 10% subset
full_dataset = True  # change to True to use full train dataset for training


# potentially take different horizons for training
def create_data_subset(data_dict):
    """
    returns last 2, 4, 6, 8 weeks of dataset
    """
    sum_timesteps = data_dict["train"].size(0)

    h_per_week = 7 * 24
    weeks_dict = {"2_weeks" : 2*h_per_week, 
                "4_weeks" : 4*h_per_week, 
                "6_weeks" : 6*h_per_week, 
                "8_weeks" : 8*h_per_week }


    for key, value in weeks_dict.items():
        data_dict[f"train_{key}"][(sum_timesteps-value):,:]     

    return data_dict

In [13]:
# use electricity dataset
data_dict = data_handling.load_electricity()

# create a smaller subset of the train dataset
if full_dataset == False:
    data_dict = create_data_subset(data_dict)

electricity = {}
electricity["dataloader_train"], electricity["dataloader_validation"], electricity["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)
data_dict["train"].shape

Feature batch shape: torch.Size([32, 96, 348])


torch.Size([4993, 348])

In [14]:
# eu electricity data
data_tensor = data_handling.eu_electricity_to_tensor()
data_dict, standardize_values =  data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# create a smaller subset of the train dataset
if full_dataset == False:
    percentage = 0.1
    ten_percent_length = int(percentage * data_dict["train"].shape[0])
    data_dict["train"] = data_dict["train"][:ten_percent_length, :]

# convert to dataloader
europe = {}
europe["dataloader_train"], europe["dataloader_validation"], europe["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)


Feature batch shape: torch.Size([32, 96, 208])


In [15]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# create a smaller subset of the train dataset
if full_dataset == False:
    percentage = 0.1
    ten_percent_length = int(percentage * data_dict["train"].shape[0])
    data_dict["train"] = data_dict["train"][:ten_percent_length, :]

# convert to datalaoder
bavaria = {}
bavaria["dataloader_train"], bavaria["dataloader_validation"], bavaria["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)

Feature batch shape: torch.Size([32, 96, 67])


In [7]:
full_dataset = True  # change to True to use full train dataset for training

# use electricity dataset
data_dict = data_handling.load_electricity()

transfer_learning_split = True

if transfer_learning_split == True:

    id_split_point = int(data_dict["train"].size(1) * 0.8)

    data_dict_source = {}
    data_dict_target = {}

    for key, value in data_dict.items():
        data_dict_source[key] = value[:,:id_split_point]
        data_dict_target[key] = value[:,id_split_point:]


    electricity = {}
    electricity["dataloader_train"], electricity["dataloader_validation"], electricity["dataloader_test"] = data_handling.convert_data(data_dict_source, window_size, pred_length)
    dataloader_train_target, dataloader_validation_target, dataloader_test_target = data_handling.convert_data(data_dict_target, window_size, pred_length)

else:
    dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

Feature batch shape: torch.Size([32, 96, 278])
Feature batch shape: torch.Size([32, 96, 70])


In [16]:
datasets = {"electricity" : electricity,
			"europe" : europe,
			"bavaria" : bavaria
            }

In [17]:
# run experiment for each normalizaiton strategie and save model and evaluation metrics

for key, dataset_dict in datasets.items():
    inputs, _ = next(iter(dataset_dict["dataloader_train"]))
    num_variates = inputs.size(2)
    
    # define parameters and create config 
    best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.2, 
                    'num_mem_tokens': 4, 'learning_rate': 0.0005}


    model_config = {
        'num_variates': num_variates,
        'lookback_len': window_size,
        'depth': best_parameters["depth"],
        'dim': best_parameters["dim"],
        'num_tokens_per_variate': 1,
        'pred_length': pred_length,
        'dim_head': best_parameters["dim_head"],
        'heads': best_parameters["heads"],
        'attn_dropout': best_parameters["attn_dropout"],
        'ff_mult': best_parameters["ff_mult"],
        'ff_dropout': best_parameters["ff_dropout"],
        'num_mem_tokens': best_parameters["num_mem_tokens"],
        'use_reversible_instance_norm': True,
        'reversible_instance_norm_affine': False,
        'flash_attn': True
    }

    # select available deviec
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 #   device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

    print(f"Using device: {device}")

    # defining all needed instances
    model = iTransformer(**model_config).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["iTransformer_baseline"] / key)

    # run model training as mentioned in the original paper
    epoch = 10

    checkpoint_path = config.CONFIG_MODEL_LOCATION["iTransformer_baseline"] / key / f"{key}_best_val_loss.pt"

    for epoch in range(1, epoch + 1):
        training_functions.train_one_epoch(epoch, model, device, dataset_dict["dataloader_train"], dataset_dict["dataloader_validation"], \
                                            optimizer, scheduler, writer, checkpoint_path)

    # load model with best validaiton mse
    checkpoint = torch.load(checkpoint_path)
    model = iTransformer(**model_config).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])

    # predict on test set
    metrics = helpers.full_eval(model, dataset_dict["dataloader_test"], device)
    for eval_metric, value in metrics[96].items():
        metrics[96][eval_metric] = value.item()


    metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
    metrics_df.rename(columns={0: key}, inplace=True)

    if full_dataset == False:
        metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{key}_epochs{epoch}_10_percent_dataset.csv")
    else:
        metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{key}_baseline.csv")


Using device: cuda


Epoch: 1:   0%|          | 0/151 [00:00<?, ?it/s]

Epoch: Validating: 100%|██████████| 21/21 [00:00<00:00, 114.92it/s]


Validation metrics: {96: {'mse': tensor(0.2528, device='cuda:0')}, 192: {'mse': 0}, 336: {'mse': 0}, 720: {'mse': 0}}
Checkpointing succesfull after epoch 1


Epoch: Validating: 100%|██████████| 21/21 [00:00<00:00, 129.15it/s]


...

Epoch: Validating: 100%|██████████| 33/33 [00:00<00:00, 298.10it/s]


Validation metrics: {96: {'mse': tensor(0.0002, device='cuda:0')}, 192: {'mse': 0}, 336: {'mse': 0}, 720: {'mse': 0}}
Checkpointing succesfull after epoch 9


Epoch: Validating: 100%|██████████| 33/33 [00:00<00:00, 308.80it/s]


Validation metrics: {96: {'mse': tensor(0.0002, device='cuda:0')}, 192: {'mse': 0}, 336: {'mse': 0}, 720: {'mse': 0}}
Checkpointing succesfull after epoch 10


Epoch: Validating: 100%|██████████| 71/71 [00:00<00:00, 265.09it/s]
